In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from tensorflow import keras
from transformers import AutoTokenizer, TFAutoModel, TFBertModel
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
import IPython

In [ ]:
# GET the data
## Memuat Data

dataset = pd.read_csv("/content/Capres2014-1.1.csv", usecols=["Isi_Tweet", "Sentimen"])

In [ ]:
# EXPLORE the data
## Menampilkan lima data terakhir
dataset.tail()

,Isi_Tweet,Sentimen
1880,Jangan kabur dari tanggung jawab dengan kemasa...,1
1881,@echo_hadiwibowo mana berani pmrntah..m G da y...,1
1882,@IndonesiaCapres ANAK MEDAN DUKUNG CAPRES JK D...,1
1883,"RT @idoidonajib: Jelek! ""@fallenokta: Apa komp...",1
1884,"Langsung deh ngadu ke capres Hatta Rajasa, bia...",1


In [ ]:
## Mengecek Imbalanced Data
dataset['Sentimen'].value_counts()

 1    1117
-1     768
Name: Sentimen, dtype: int64

In [ ]:
## Mengganti {-1,1} menjadi {0,1}
dataset['Sentimen'] = dataset['Sentimen'].replace(-1,0)

dataset['Sentimen'].value_counts()

1    1117
0     768
Name: Sentimen, dtype: int64

In [ ]:
# MODEL the data
## Pra Pengolahan - Cleaning

def clean_text(tweet):
    
    # Convert to lower case
    tweet = tweet.lower()
    # remove unicode characters
    tweet = tweet.encode('ascii', 'ignore').decode()
    # Clean www.* or https?://*
    tweet = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))','',tweet)
    # Clean @username
    tweet = re.sub(r'@[^\s]+','',tweet)
    #Remove additional white spaces
    tweet = re.sub(r'[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    
    #trim
    tweet = tweet.strip('\'"')
    
    return tweet

dataset["Isi_Tweet"] = dataset['Isi_Tweet'].map(lambda x: clean_text(x))
dataset = dataset[dataset['Isi_Tweet'].apply(lambda x: len(x.split()) >=1)]
dataset.shape

(1885, 2)

In [ ]:
## Pra Pengolahan - Splitting

train_data, test_data, train_labels, test_labels = train_test_split(
    dataset['Isi_Tweet'], dataset['Sentimen'], test_size=0.2, random_state=42)

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 79.2 MB/s eta 0:00:00


In [ ]:
bert_tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p2")
def tokenisasi(teks):
    encode_dict = bert_tokenizer(teks,
                                   add_special_tokens = True,
                                   max_length = 128, 
                                   padding = 'max_length',
                                   truncation = True,
                                   return_attention_mask = True,
                                   return_tensors = 'tf',)

    tokenID = encode_dict['input_ids']
    attention_mask = encode_dict['attention_mask']

    return tokenID, attention_mask

def create_input(data):
    tokenID, input_mask = [], []
    for teks in data:
        token, mask = tokenisasi(teks)
        tokenID.append(token)
        input_mask.append(mask)
    
    return [np.asarray(tokenID, dtype=np.int32).reshape(-1, 128), 
            np.asarray(input_mask, dtype=np.int32).reshape(-1, 128)]

bert_model = TFAutoModel.from_pretrained("indobenchmark/indobert-base-p2", trainable=False)
def bert(hp):
    
    #Input layer
    input_token = keras.layers.Input(shape=(128,), dtype=np.int32,
                                        name="input_token")
    input_mask = keras.layers.Input(shape=(128,), dtype=np.int32,
                                   name="input_mask")

    #hidden layer
    hid_layer = hp.Choice('hid_layer_type', ['lstm', 'dense', 'both', 'none'])

    # hidden layer dense (bert embedding -> dense -> output)
    if hid_layer == 'dense':
        bert_embedding = bert_model([input_token, input_mask])[1]
        n_hid_layer = hp.Choice('n_layer', [1, 2])

        if n_hid_layer == 1:
            hidden_0 = keras.layers.Dense(hp.Int('neuron_0', 32, 128), activation='relu',
                                        kernel_regularizer=keras.regularizers.l2(hp.Float('l2_hidden_0', .001, .1)))(bert_embedding) 
            output = keras.layers.Dense(1, activation='sigmoid',
                                        kernel_regularizer=keras.regularizers.l2(hp.Float('l2_output', .001, .1)))(hidden_0)  
        else:
            hidden_0 = keras.layers.Dense(hp.Int('neuron_0', 32, 128), activation='relu',
                                        kernel_regularizer=keras.regularizers.l2(hp.Float('l2_hidden_0', .001, .1)))(bert_embedding)
            hidden_1 = keras.layers.Dense(hp.Int('neuron_1', 32, 128), activation='relu',
                                        kernel_regularizer=keras.regularizers.l2(hp.Float('l2_hidden_1', .001, .1)))(hidden_0) 
            output = keras.layers.Dense(1, activation='sigmoid',
                                        kernel_regularizer=keras.regularizers.l2(hp.Float('l2_output', .001, .1)))(hidden_1)
    
    # hidden layer lstm (bert embedding -> lstm -> output)
    elif hid_layer == 'lstm':
        bert_embedding = bert_model([input_token, input_mask])[0]
        bidirectional = hp.Choice('lstm_bidirectional', [True, False])

        if bidirectional:
            lstm = keras.layers.Bidirectional(CuDNNLSTM(hp.Int('lstm_neuron', 32, 128),
                                                        kernel_regularizer=keras.regularizers.l2(hp.Float('l2_lstm', .001, .1))))(bert_embedding)
        else:
            lstm = CuDNNLSTM(hp.Int('lstm_neuron', 32, 128),
                             kernel_regularizer=keras.regularizers.l2(hp.Float('l2_lstm', .001, .1)))(bert_embedding)
        output = keras.layers.Dense(1, activation='sigmoid',
                                    kernel_regularizer=keras.regularizers.l2(hp.Float('l2_output', .001, .1)))(lstm)

    # hidden layer lstm + dense (bert embedding -> lstm -> dense -> output)
    elif hid_layer == 'both':
        bert_embedding = bert_model([input_token, input_mask])[0]
        lstm = keras.layers.Bidirectional(CuDNNLSTM(hp.Int('lstm_neuron', 32, 128),
                                                    kernel_regularizer=keras.regularizers.l2(hp.Float('l2_lstm', .001, .1))))(bert_embedding)
        hidden_0 = keras.layers.Dense(hp.Int('neuron_0', 32, 128), activation='relu',
                                      kernel_regularizer=keras.regularizers.l2(hp.Float('l2_hidden_0', .001, .1)))(lstm)
        output = keras.layers.Dense(1, activation='sigmoid',
                                    kernel_regularizer=keras.regularizers.l2(hp.Float('l2_output', .001, .1)))(hidden_0)

    # no hiddden layer (bert embedding -> output)
    else:
        bert_embedding = bert_model([input_token, input_mask])[1]
        output = keras.layers.Dense(1, activation='sigmoid',
                                    kernel_regularizer=keras.regularizers.l2(hp.Float('l2_output', .001, .1)))(bert_embedding)

    
    model = keras.models.Model(inputs=[input_token, input_mask], outputs=output)

    model.compile(optimizer=keras.optimizers.Adam(hp.Float('lr', 1e-3, 5e-2)), loss='binary_crossentropy', metrics=['accuracy'])
   
    return model

class ClearTrainingOutput(keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,
                                           restore_best_weights=True)

Some layers from the model checkpoint at indobenchmark/indobert-base-p2 were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at indobenchmark/indobert-base-p2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.8 MB/s eta 0:00:00


In [ ]:
from keras_tuner.tuners import BayesianOptimization

bert_train_data = create_input(train_data)
bert_test_data = create_input(test_data)

tuner = BayesianOptimization(bert,
                             objective = 'val_accuracy', 
                             max_trials = 25,
                             directory = 'tune',
                             project_name = 'Sentiment-BERT',
                             overwrite = True)

tuner.search(bert_train_data, train_labels,
             batch_size=256, epochs=50,
             validation_data=(bert_test_data, test_labels),
             callbacks=[early_stop, ClearTrainingOutput()])

Trial 25 Complete [00h 07m 09s]
val_accuracy: 0.8275862336158752

Best val_accuracy So Far: 0.8514589071273804
Total elapsed time: 03h 55m 51s


In [ ]:
tuner.results_summary()

Results summary
Results in tune/Sentiment-BERT
Showing 10 best trials
Trial summary
Hyperparameters:
hid_layer_type: lstm
lstm_bidirectional: 1
lstm_neuron: 127
l2_lstm: 0.0032251716258970705
l2_output: 0.06278849212607956
lr: 0.005534638759858194
Score: 0.8514589071273804
Trial summary
Hyperparameters:
hid_layer_type: lstm
lstm_bidirectional: 1
lstm_neuron: 50
l2_lstm: 0.021584928051188775
l2_output: 0.09196880065556831
lr: 0.011835662468733413
neuron_0: 77
l2_hidden_0: 0.045830864494719986
Score: 0.8381962776184082
Trial summary
Hyperparameters:
hid_layer_type: both
lstm_bidirectional: 1
lstm_neuron: 44
l2_lstm: 0.06044418802496354
l2_output: 0.015050653620328022
lr: 0.007628435985180532
neuron_0: 103
l2_hidden_0: 0.005753463232814367
n_layer: 2
Score: 0.8328912258148193
Trial summary
Hyperparameters:
hid_layer_type: lstm
lstm_bidirectional: 1
lstm_neuron: 127
l2_lstm: 0.01795837346561108
l2_output: 0.06991510565543062
lr: 0.011357654921975456
neuron_0: 106
l2_hidden_0: 0.04821678446

In [ ]:
# Train with higher epochs
input_token = keras.layers.Input(shape=(128,), dtype=np.int32,
                                 name="input_token")
input_mask = keras.layers.Input(shape=(128,), dtype=np.int32,
                                name="input_mask")

bert_model = TFAutoModel.from_pretrained("indobenchmark/indobert-base-p2", trainable=False)
bert_embedding = bert_model([input_token, input_mask])[0]

lstm = keras.layers.Bidirectional(CuDNNLSTM(127, kernel_regularizer=keras.regularizers.l2(0.00322)))(bert_embedding)

output = keras.layers.Dense(1, activation='sigmoid', 
                            kernel_regularizer=keras.regularizers.l2(0.06278))(lstm)

model = keras.models.Model(inputs=[input_token, input_mask], outputs=output)

model.compile(optimizer=keras.optimizers.Adam(0.00553),
               loss='binary_crossentropy',
               metrics=['accuracy'])

model.summary()

Some layers from the model checkpoint at indobenchmark/indobert-base-p2 were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at indobenchmark/indobert-base-p2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model_7 (TFBertModel)  TFBaseModelOutputWi  124441344   ['input_token[0][0]',            
                                thPoolingAndCrossAt               'input_mask[0][0]']             
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                         

In [ ]:
bert_train_data = create_input(train_data)
bert_test_data = create_input(test_data)

early_stop = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20,
                                           restore_best_weights=True)

model.fit(bert_train_data, train_labels, epochs=100, batch_size=256,
           validation_data=(bert_test_data, test_labels),
           callbacks=[early_stop])

Epoch 1/100
6/6 [==============================] - 39s 4s/step - loss: 4.0657 - accuracy: 0.6346 - val_loss: 2.6108 - val_accuracy: 0.6286
Epoch 2/100
6/6 [==============================] - 20s 3s/step - loss: 2.0989 - accuracy: 0.7288 - val_loss: 1.5717 - val_accuracy: 0.7931
Epoch 3/100
6/6 [==============================] - 20s 3s/step - loss: 1.3921 - accuracy: 0.7898 - val_loss: 1.1896 - val_accuracy: 0.7958
Epoch 4/100
6/6 [==============================] - 20s 3s/step - loss: 1.0917 - accuracy: 0.8031 - val_loss: 0.9647 - val_accuracy: 0.8302
Epoch 5/100
6/6 [==============================] - 20s 3s/step - loss: 0.8613 - accuracy: 0.8375 - val_loss: 0.7913 - val_accuracy: 0.8249
Epoch 6/100
6/6 [==============================] - 20s 3s/step - loss: 0.6938 - accuracy: 0.8561 - val_loss: 0.6780 - val_accuracy: 0.8355
Epoch 7/100
6/6 [==============================] - 20s 4s/step - loss: 0.5774 - accuracy: 0.8846 - val_loss: 0.6241 - val_accuracy: 0.8462
Epoch 8/100
6/6 [==========

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/model_mlp_sentiment_8806.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/model_mlp_sentiment_8806.h5',
                                 custom_objects={"TFBertModel": TFBertModel})

model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model_10 (TFBertModel)  TFBaseModelOutputWi  124441344  ['input_token[0][0]',            
                                thPoolingAndCrossAt               'input_mask[0][0]']             
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                         

In [ ]:
test_loss, test_acc = model.evaluate(bert_test_data, test_labels)
print('Test accuracy:', test_acc)

12/12 [==============================] - 7s 290ms/step - loss: 0.5656 - accuracy: 0.8806
Test accuracy: 0.8806366324424744
